<a href="https://colab.research.google.com/github/jukdang/AI_repo/blob/main/Dacon_toursism%20data%20classification/Union.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gluonnlp
!pip install mxnet
!pip install sentencepiece
!pip install transformers
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master
!pip install efficientnet_pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-xd5qskw3
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-xd5qskw3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
#!pip install torch==1.10.1+cu111 torchvision==0.11.2+cu111 torchaudio==0.10.1 #-f https://download.pytorch.org/whl/torch_stable.html

In [1]:
#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#efficientNet
from efficientnet_pytorch import EfficientNet

In [2]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

import torchvision
from torchvision import transforms

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, IterableDataset, DataLoader
from tqdm import tqdm, tqdm_notebook

import gluonnlp as nlp

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
cd ./drive/MyDrive/

/content/drive/MyDrive


In [5]:
device = torch.device("cuda:0")

n_devices = torch.cuda.device_count()
print(n_devices)

for i in range(n_devices):
    print(torch.cuda.get_device_name(i))

if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

1
Tesla T4
There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [6]:
bertmodel, vocab = get_pytorch_kobert_model()
efficientnet = EfficientNet.from_pretrained('efficientnet-b4')

using cached model. /content/drive/MyDrive/.cache/kobert_v1.zip
using cached model. /content/drive/MyDrive/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
Loaded pretrained weights for efficientnet-b4


In [7]:
train_df = pd.read_csv('train_processed2.csv')
train_df['drive_img_path'] = './' + train_df['resized_img_path'].str[8:]
train_df

,id,resized_img_path,overview,sentence_length,category1,category2,category3,tokenized_overview,drive_img_path
0,TRAIN_00000,./image/resized_train/TRAIN_00000.jpg,소안항은 조용한 섬으로 인근해안이 청정해역으로 일찍이 김 양식을 해서 높은 소득을 ...,217,0,0,0,"['소안항', '은', '조용한', '섬', '으로', '인근', '해안이', '청...",./resized_train/TRAIN_00000.jpg
1,TRAIN_00001,./image/resized_train/TRAIN_00001.jpg,경기도 이천시 모가면에 있는 골프장으로 대중제 18홀이다. 회원제로 개장을 했다가 ...,244,1,1,1,"['경기도', '이천시', '모가면', '에', '있는', '골프장', '으로', ...",./resized_train/TRAIN_00001.jpg
2,TRAIN_00002,./image/resized_train/TRAIN_00002.jpg,금오산성숯불갈비는 한우고기만을 전문적으로 취급하고 사용하는 부식 자재 또한 유기농법...,160,2,2,2,"['금오산성', '숯불', '갈비', '는', '한우', '고기', '만을', '전...",./resized_train/TRAIN_00002.jpg
3,TRAIN_00003,./image/resized_train/TRAIN_00003.jpg,철판 위에서 요리하는 안동찜닭을 맛볼 수 있는 곳이다. 경상북도 안동시에 있는 한식...,67,2,2,2,"['철판', '위', '에서', '요리', '하는', '안동찜닭', '을', '맛볼...",./resized_train/TRAIN_00003.jpg
4,TRAIN_00004,./image/resized_train/TRAIN_00004.jpg,※ 영업시간 10:30 ~ 20:30\n\n3대에 걸쳐 아귀만을 전문으로 취급하는 ...,63,2,2,2,"['※', '영업', '시간', '10:30', '~', '20:30', '\n\n...",./resized_train/TRAIN_00004.jpg
...,...,...,...,...,...,...,...,...,...
16981,TRAIN_16981,./image/resized_train/TRAIN_16981.jpg,해발 12000m에 자리한 식담겸 카페점문점이다.<br>곤드레밥과 감자전을 판매하고...,50,2,2,2,"['해발', '12000', 'm', '에', '자리', '한', '식담', '겸'...",./resized_train/TRAIN_16981.jpg
16982,TRAIN_16982,./image/resized_train/TRAIN_16982.jpg,설악힐호텔은 동해고속도로 속초톨게이트에서 멀지 않은 관광로 변에 있다. 속초의 대표...,471,4,7,11,"['설악힐', '호텔', '은', '동해', '고속도로', '속초', '톨게이트',...",./resized_train/TRAIN_16982.jpg
16983,TRAIN_16983,./image/resized_train/TRAIN_16983.jpg,충남 서산시 중심가에 위치한 줌모텔은 프라이버스가 보장되는 조용한 공간으로 가치가 ...,282,4,7,11,"['충남', '서산시', '중심', '가에', '위치', '한', '줌', '모텔'...",./resized_train/TRAIN_16983.jpg
16984,TRAIN_16984,./image/resized_train/TRAIN_16984.jpg,토토큰바위캠핑장은 경기도 가평지역 내에서도 청정지역으로 손꼽히는 지역으로 주변에 화...,158,1,1,3,"['토', '토큰', '바위', '캠핑장', '은', '경기도', '가평', '지역...",./resized_train/TRAIN_16984.jpg


In [8]:
num_classes = len(train_df['category3'].unique())

In [9]:
train, test = train_test_split(train_df[['overview', 'drive_img_path', 'category3']])
train_df = train.to_numpy()
test_df = test.to_numpy()

In [10]:
class InputIterableDataset(IterableDataset):
    def __init__(self, dataset, sentence_idx=0, img_idx=1, label_idx=2, bert_tokenizer=None, max_len=400, pad=True, pair=False):
        self.dataset = dataset
        self.transform_sentence = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.transform_image = transforms.Compose([transforms.Resize((224,224)), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

    def __iter__(self):
      for overview, img_path, label in self.dataset:
        sentence = self.transform_sentence(overview)
        img = self.transform_image(torchvision.io.read_image(img_path).type(torch.float32))
        label = np.int32(label)
        yield (sentence, img, label)

In [11]:
# Setting parameters 필수
max_len = 400
batch_size = 10
warmup_ratio = 0.1
num_epochs = 15
max_grad_norm = 1
log_interval = 399
learning_rate =  5e-5

In [12]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/drive/MyDrive/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [13]:
data_train = InputIterableDataset(train_df, 0, 1, 2, tok, max_len, True, False)
data_test = InputIterableDataset(test_df, 0, 1, 2, tok, max_len, True, False)

train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [14]:
class Classifier(nn.Module):
    def __init__(self,
                 eff,
                 bert,
                 bert_hidden_size = 768,
                 eff_hidden_size = 1000,
                 num_classes = num_classes,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(Classifier, self).__init__()
        self.eff = eff
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(bert_hidden_size + eff_hidden_size, num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids, image):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, bert_out = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        eff_out = self.eff(image)

        if self.dr_rate:
            bert_out = self.dropout(bert_out)
            eff_out = self.dropout(eff_out)
        
        cat = torch.cat([bert_out, eff_out], dim=1)
        out = self.classifier(cat)

        return out

In [15]:
model = Classifier(efficientnet, bertmodel,  dr_rate=0.5).to(device)

In [ ]:
#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_function = nn.CrossEntropyLoss()

t_total = len(train_df) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (sentence, img, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()

        token_ids, valid_length, segment_ids = sentence
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length.to(device)
        img = img.float().to(device)
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids, img)

        loss = loss_function(out, label)
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule

        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))

        del img, label, out
        torch.cuda.empty_cache()

    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    
    model.eval()
    with torch.no_grad():
      for batch_id, (sentence, img, label) in enumerate(tqdm_notebook(test_dataloader)):
          token_ids, valid_length, segment_ids = sentence
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)
          valid_length= valid_length.to(device)
          img = img.float().to(device)
          label = label.long().to(device)

          out = model(img)
          
          test_acc += calc_accuracy(out, label)

          del img, label, out
          torch.cuda.empty_cache()
          
      print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

epoch 1 batch id 1 loss 4.87202787399292 train acc 0.0
epoch 1 batch id 400 loss 5.458861351013184 train acc 0.01125
epoch 1 batch id 799 loss 4.568451881408691 train acc 0.027284105131414275
epoch 1 batch id 1198 loss 3.8356289863586426 train acc 0.08005008347245415
epoch 1 batch id 1597 loss 3.395037889480591 train acc 0.11546649968691267
epoch 1 batch id 1996 loss 3.3688812255859375 train acc 0.14358717434869692
epoch 1 batch id 2395 loss 3.7434113025665283 train acc 0.16676409185803712
epoch 1 batch id 2794 loss 3.018500804901123 train acc 0.18550465282748693
epoch 1 batch id 3193 loss 3.708665132522583 train acc 0.19890385217663537
epoch 1 batch id 3592 loss 2.3511149883270264 train acc 0.21208240534520997
epoch 1 batch id 3991 loss 4.134548664093018 train acc 0.22327737409170423
epoch 1 batch id 4390 loss 3.5182204246520996 train acc 0.2314123006833689
epoch 1 batch id 4789 loss 2.816399097442627 train acc 0.24140739193986027
epoch 1 batch id 5188 loss 2.6308584213256836 train ac